# Predict Winner from Realtime Stats using RNN

In [64]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.autograd import Variable
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm


SEED = 233
BATCH_SIZE = 32
PREDICT_TIME = 5

## Dataset

In [65]:
class LOLDataset(Dataset):
    def __init__(self,moneyData,label):
        self.money=[]
        for i in range(PREDICT_TIME):
            self.money.append([line[i] for line in moneyData])
        self.label=[i for i in label]
    
    def __getitem__(self,item):
        return torch.tensor([[torch.scalar_tensor(x[item])] for x in self.money]),torch.tensor(self.label[item])
    
    def __len__(self):
        return len(self.label)
            

In [66]:
df = pd.read_csv('../data/LeagueofLegends.csv', sep=',')
df = df[df['gamelength'] >= PREDICT_TIME]
df.reset_index(drop = True, inplace = True)
print(f'# of matches: {len(df)}')

from ast import literal_eval
df['golddiff'] = df['golddiff'].apply(literal_eval)
df['golddiff'].head()
# df["bResult"]

# of matches: 7620


0    [0, 0, -14, -65, -268, -431, -488, -789, -494,...
1    [0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...
2    [0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...
3    [0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...
4    [40, 40, 44, -36, 113, 158, -121, -191, 23, 20...
Name: golddiff, dtype: object

Normalize data:

In [67]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for row in df['golddiff']:
    scaler.partial_fit(np.asanyarray(row).reshape(-1, 1))

golddiff = [scaler.transform(np.asanyarray(row).reshape(-1, 1)).reshape(-1) for row in df['golddiff']]
print(len(golddiff))

7620


Split the dataset into Train:Held-Out:Test = 6:2:2.

In [68]:
dataset = LOLDataset(golddiff, df["bResult"])
test_size = valid_size = int(0.2 * len(dataset))
train_size = len(dataset)- test_size - valid_size

trainDataset, validDataset, testDataset = random_split(
    dataset = dataset,
    lengths = [train_size, valid_size, test_size],
    generator = torch.Generator().manual_seed(SEED)
)

trainLoader = DataLoader(trainDataset, batch_size = BATCH_SIZE, shuffle=True)
validLoader = DataLoader(validDataset, batch_size = BATCH_SIZE)
testLoader = DataLoader(testDataset, batch_size = BATCH_SIZE)

## RNN Network

In [69]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN,self).__init__()

        self.rnn=nn.LSTM(
            input_size = 1,
            hidden_size = 256,
            num_layers = 1,
            batch_first = True
        )

        self.out = nn.Linear(256, 2)
    
    def forward(self,x):
        r_out,(h_n, h_c)=self.rnn(x,None)
        out = self.out(r_out[:, -1, :])
        return out

In [70]:
model=RNN()
print(model)

RNN(
  (rnn): LSTM(1, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)


## Train

In [71]:
def train(dataloader, model, loss_fn, optimizer, mute = False):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        x, y = Variable(x), Variable(y)

        predict = model(x)
        loss = loss_fn(predict, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 20 == 0 and not mute:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [72]:
def test(dataloader, model, loss_fn, validation = False):
    model.eval()
    size = len(dataloader.dataset)

    correct = 0
    test_loss = 0
    with torch.no_grad():
        for step,(x,y) in enumerate(dataloader):
            x, y = Variable(x), Variable(y)
            predict = model(x)
            # print(predict)
            test_loss += loss_fn(predict, y).item()
            correct += (predict.argmax(1) == y).sum().item()
    
    print(f"{'Valid' if validation else 'Test'} Acc:{correct/size:>7f}, Avg Loss: {test_loss/size:>7f}")

In [73]:
MUTE = False
EPOCH = 15
LR = 0.0001

torch.manual_seed(SEED)
np.random.seed(SEED)

optimizer = torch.optim.Adam(model.parameters(), lr = LR)
loss_func = nn.CrossEntropyLoss()

for epoch in range(1, EPOCH + 1):
    print(f"--------- Epoch #{epoch} ---------")
    train(trainLoader, model, loss_func, optimizer, mute = MUTE)
    test(validLoader, model, loss_func, validation = True)
    # test(testLoader, model, loss_func)


--------- Epoch #1 ---------
loss: 0.687814  [    0/ 4572]
loss: 0.683205  [  640/ 4572]
loss: 0.691192  [ 1280/ 4572]
loss: 0.676130  [ 1920/ 4572]
loss: 0.701419  [ 2560/ 4572]
loss: 0.679458  [ 3200/ 4572]
loss: 0.671152  [ 3840/ 4572]
loss: 0.720431  [ 4480/ 4572]
Valid Acc:0.554462, Avg Loss: 0.021643
--------- Epoch #2 ---------
loss: 0.690452  [    0/ 4572]
loss: 0.696368  [  640/ 4572]
loss: 0.676539  [ 1280/ 4572]
loss: 0.679764  [ 1920/ 4572]
loss: 0.696806  [ 2560/ 4572]
loss: 0.690974  [ 3200/ 4572]
loss: 0.701068  [ 3840/ 4572]
loss: 0.691150  [ 4480/ 4572]
Valid Acc:0.554462, Avg Loss: 0.021628
--------- Epoch #3 ---------
loss: 0.696183  [    0/ 4572]
loss: 0.679740  [  640/ 4572]
loss: 0.666869  [ 1280/ 4572]
loss: 0.680429  [ 1920/ 4572]
loss: 0.683630  [ 2560/ 4572]
loss: 0.714930  [ 3200/ 4572]
loss: 0.679446  [ 3840/ 4572]
loss: 0.679076  [ 4480/ 4572]
Valid Acc:0.554462, Avg Loss: 0.021626
--------- Epoch #4 ---------
loss: 0.686875  [    0/ 4572]
loss: 0.697519  [

## Test

In [74]:
test(testLoader, model, loss_func)

Test Acc:0.585302, Avg Loss: 0.021080
